In [1]:
import os
import re
import numpy as np
import pandas as pd

In [2]:
folder_path = "data"
file_name   = "preco_medicamento.csv"
path = os.path.join(folder_path,file_name)

### **Ler o arquivo** 

<details><summary><b>SPOILER (Resposta para continuar o exercicio)</b></summary>
    
```python
df = pd.read_csv(path, encoding="latin-1", sep=";")
```

Obs: comando ```file``` pode trazer informações interessantes sobre o problema.

In [3]:
df = pd.read_csv(path, encoding="latin-1", sep = ";")
df.head(1)

,SUBSTÂNCIA,PF 17% ALC,PF 17,TIPO DE PRODUTO,COMERCIALIZAÇÃO 2019
0,CLORIDRATO DE MEMANTINA,"155,54","155,54",Similar,Sim


### **Arrumar o Head (Opcional)** 

In [4]:
old_column_names = df.columns.copy()

In [5]:
old_column_names

Index(['SUBSTÂNCIA', 'PF 17% ALC', 'PF 17', 'TIPO DE PRODUTO',
       'COMERCIALIZAÇÃO 2019'],
      dtype='object')

In [6]:
df.columns = map(str.lower, df.columns)

In [7]:
df.columns = map(lambda x: x.replace(" ", "_"),df.columns)
df.columns = map(lambda x: x.replace("%", ""),df.columns)

### **Checar dataframe**

1. Mostrar a quantidade de linhas e colunas.
2. Mostrar o tipo de dado de cada coluna.
3. Mostrar a quantidade de dados faltantes em cada coluna. Se houver, remover  as linhas faltantes.
4. Checar existência de linhas duplicadas (Caso exista remover).

**Parte 1**

In [8]:
print(f"Qtd Linhas: {df.shape[0]}\nQtd Colunas: {df.shape[1]}")

Qtd Linhas: 24291
Qtd Colunas: 5


**Parte 2**

In [9]:
df.dtypes

substância              object
pf_17_alc               object
pf_17                   object
tipo_de_produto         object
comercialização_2019    object
dtype: object

**Parte 3**

In [10]:
#mostra se tem ou não dado faltante.
print(df.isna().any())
print("*"*20)
#conta a quantidade de dados faltantes.
print(df.isna().apply(lambda x: sum(x)))
print("*"*20)
#faz a memsa coisa mais simples.
print(df.isna().sum(axis=0)) #axis=0 é default
print("*"*20)
#Remove as linhas
df.dropna(inplace = True)

substância              False
pf_17_alc                True
pf_17                    True
tipo_de_produto         False
comercialização_2019    False
dtype: bool
********************
substância                0
pf_17_alc               772
pf_17                   772
tipo_de_produto           0
comercialização_2019      0
dtype: int64
********************
substância                0
pf_17_alc               772
pf_17                   772
tipo_de_produto           0
comercialização_2019      0
dtype: int64
********************


**Parte 4**

In [11]:
cond = df.duplicated()
if len(df[cond]):
    print("Existem dados faltantes.")
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True,inplace=True)
else:
    print("Não existem dados faltantes.")

Existem dados faltantes.


### **Estatisticas das variáveis** 

1. Calcular média e desvio padrão para as variáveis "PF 17% ALC" e "PF 17"
2. Na variável comercialização_2019 mapear para 1 quando for "Sim" e 0 quando for "Não". Caso seja outro valor dropar a linha.
3. Calcular a média e desvio padrão para as variáveis "PF 17% ALC" e "PF 17"

**Parte 1**

In [12]:
df["pf_17_alc"] = df["pf_17_alc"].apply(lambda x:x.replace(",",".")).astype(float)

df["pf_17"] = df["pf_17"].apply(lambda x:x.replace(",",".")).astype(float)

In [13]:
print("Média:")
print(df[["pf_17_alc","pf_17"]].mean())
print("\nStd")
print(df[["pf_17_alc","pf_17"]].std())
########################################
get_mean_std = lambda series: (series.mean(),series.std()) 
get_mean_std(df["pf_17_alc"])

Média:
pf_17_alc    836.399889
pf_17        883.130541
dtype: float64

Std
pf_17_alc    7198.428926
pf_17        7640.557035
dtype: float64


(836.3998885807191, 7198.428925730309)

**Parte 2**

In [14]:
df["comercialização_2019"].value_counts()

Sim     11095
NÃ£o     9816
???         1
Name: comercialização_2019, dtype: int64

In [15]:
idx_to_remove = df[df["comercialização_2019"] == "???"].index[0]

In [16]:
print(f"Antes: {len(df)}")
df.drop(idx_to_remove, inplace=True)
df.reset_index(drop=True, inplace=True)
print(f"Depois: {len(df)}")

Antes: 20912
Depois: 20911



Uma das maneiras:
```python
cond_yes = df["comercialização_2019"] == "Sim"
cond_no  = df["comercialização_2019"] == "NÃ£o"

df.loc[cond_yes,"comercialização_2019"] = 1
df.loc[cond_no,"comercialização_2019"]  = 0
```

In [17]:
df["comercialização_2019"] = df["comercialização_2019"].apply(lambda x: 1 if x == "Sim" else 0)

In [18]:
df["comercialização_2019"].value_counts()

1    11095
0     9816
Name: comercialização_2019, dtype: int64

**Parte 3**

Melhor forma de resolver esse problema:

```python 
df.groupby(by="comercialização_2019").agg({"pf_17_alc":["mean","std"], "pf_17":["mean","std"]})
```